# title을 예측하는 model 생성

In [1]:
# 내 드라이브에 대한 주소
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
# konlpy Mecab 사용하기

!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 21.4MB/s 
     |████████████████████████████████| 460kB 48.6MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
path = "/gdrive/My Drive/"

train = pd.read_csv(path+"news_train.csv")
#test = pd.read_csv(path+"news_test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

In [7]:
print(train.shape)
#print(test.shape)

(118745, 6)


# 텍스트 전처리

In [16]:
from konlpy.tag import Mecab
import re
from konlpy.tag import Okt

"""'로','으로', '다', '했', '에', '의', '에서', '부터', '아', '하','고','도','것','그','으로','해진
['을', '를', '이', '가', '은', '는', 'null','부터','에','에서','하','고','으로','로','의','만','하','고','도','았','었','다'"""

def text_preprocessing(text_list):
    
    stopwords = [] #불용어 설정
    
    tokenizer = Mecab() #형태소 분석기 
    token_list = [] 
    
    for text in text_list:
        txt = re.sub('[^가-힣]', ' ', text) #한글, 영어만 남기고 다른 글자 모두 제거
        txt = re.sub('[가-힣\s]+기자]','기자', txt) #기자 이름 제거
        token = tokenizer.morphs(txt) #형태소 분석

        #형태소 분석 결과 중 stopwords에 해당하지 않고, float type이 아닌 것만 수집
        token = [t for t in token] 
        token_list.append(token)
        
    return token_list, tokenizer

#형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 되기 때문입니다. 
train['new_article'], mecab = text_preprocessing(train['content'])
#title도 동일하게 진행
train['new_title'], title_mecab = text_preprocessing(train['title'])

In [17]:
# 결측치 제거
train = train[train["new_article"].apply(lambda x: False if len(x)==0 else True)]

# Vectorization

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 40
title_max_len = 20

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len, truncating="post") # 길이를 맞춰줌
    return X_train, vocab_size, tokenizer

train_y = train['info']
train_X, vocab_size, vectorizer = text2sequence(train['new_article'], max_len = max_len)
title_X, title_vocab_size, title_vectorizer = text2sequence(train['new_title'], max_len = title_max_len)

print(train_X.shape, train_y.shape)

vocab_size :  33461
vocab_size :  7926
(118414, 40) (118414,)


# word2vec

In [19]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
path = "/gdrive/My Drive/"

In [20]:
# 한국어 word2vec model
word2vec = gensim.models.Word2Vec.load(path+'ko.bin')
embedding_size = 200

In [21]:
embedding_matrix = np.zeros((vocab_size, 200))
title_embedding_matrix = np.zeros((title_vocab_size, 200))
vocab = vectorizer.word_index
title_vocab = title_vectorizer.word_index
count = 0

for idx, word in enumerate(vocab):
    if word in word2vec:
      embedding_vector = word2vec[word]
      embedding_matrix[idx] = embedding_vector
    else: # 임베딩 모델에 없는 것
      print(word, "word2vec에 없는 단어입니다.")
      count += 1
      pass


for idx, word in enumerate(title_vocab):
    if word in word2vec:
      embedding_vector = word2vec[word]
      title_embedding_matrix[idx] = embedding_vector
    else: # 임베딩 모델에 없는 것
      print(word, "word2vec에 없는 단어입니다.")
      count += 1
      pass
      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
네오플 word2vec에 없는 단어입니다.
스트랜드 word2vec에 없는 단어입니다.
사이버펑크 word2vec에 없는 단어입니다.
김국 word2vec에 없는 단어입니다.
애저 word2vec에 없는 단어입니다.
일회 word2vec에 없는 단어입니다.
양쯔 word2vec에 없는 단어입니다.
김선중 word2vec에 없는 단어입니다.
체임버 word2vec에 없는 단어입니다.
등가성 word2vec에 없는 단어입니다.
결정계수 word2vec에 없는 단어입니다.
셰일가스 word2vec에 없는 단어입니다.
밀어붙였 word2vec에 없는 단어입니다.
사빈 word2vec에 없는 단어입니다.
권리관계 word2vec에 없는 단어입니다.
축연 word2vec에 없는 단어입니다.
위메이드 word2vec에 없는 단어입니다.
앞지른 word2vec에 없는 단어입니다.
전력망 word2vec에 없는 단어입니다.
내려감 word2vec에 없는 단어입니다.
홍재성 word2vec에 없는 단어입니다.
이원우 word2vec에 없는 단어입니다.
시모네 word2vec에 없는 단어입니다.
다년 word2vec에 없는 단어입니다.
스크립스 word2vec에 없는 단어입니다.
이현철 word2vec에 없는 단어입니다.
배재훈 word2vec에 없는 단어입니다.
스탠더 word2vec에 없는 단어입니다.
박성광 word2vec에 없는 단어입니다.
이솔 word2vec에 없는 단어입니다.
고준희 word2vec에 없는 단어입니다.
아찔 word2vec에 없는 단어입니다.
등라 word2vec에 없는 단어입니다.
배도환 word2vec에 없는 단어입니다.
속사정 word2vec에 없는 단어입니다.
이주형 word2vec에 없는 단어입니다.
노은식 word2vec에 없는 단어입니다.
조촐 word2vec에 없는 단어입니다.
하갈동 word2vec에 없는 단어입니다.
일우 word2vec에 없는 단어입니다.
함꼐 word

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



고주리 word2vec에 없는 단어입니다.
황산화물 word2vec에 없는 단어입니다.
남친 word2vec에 없는 단어입니다.
신민철 word2vec에 없는 단어입니다.
뚫렸 word2vec에 없는 단어입니다.
팔린 word2vec에 없는 단어입니다.
조항조 word2vec에 없는 단어입니다.
포스코 word2vec에 없는 단어입니다.
급반등 word2vec에 없는 단어입니다.
일물 word2vec에 없는 단어입니다.
앤디 word2vec에 없는 단어입니다.
웃돈 word2vec에 없는 단어입니다.
나선다 word2vec에 없는 단어입니다.
평택 word2vec에 없는 단어입니다.
위클리 word2vec에 없는 단어입니다.
디톡스 word2vec에 없는 단어입니다.
한샘 word2vec에 없는 단어입니다.
앤지 word2vec에 없는 단어입니다.
맛집 word2vec에 없는 단어입니다.
재택근무 word2vec에 없는 단어입니다.
불붙 word2vec에 없는 단어입니다.
살린다 word2vec에 없는 단어입니다.
구내식당 word2vec에 없는 단어입니다.
휴무 word2vec에 없는 단어입니다.
대숲 word2vec에 없는 단어입니다.
정세랑 word2vec에 없는 단어입니다.
으로부터 word2vec에 없는 단어입니다.
아바코 word2vec에 없는 단어입니다.
행정소송 word2vec에 없는 단어입니다.
전달식 word2vec에 없는 단어입니다.
가져 word2vec에 없는 단어입니다.
관리관 word2vec에 없는 단어입니다.
투증 word2vec에 없는 단어입니다.
파미 word2vec에 없는 단어입니다.
벽장 word2vec에 없는 단어입니다.
불연 word2vec에 없는 단어입니다.
이종필 word2vec에 없는 단어입니다.
불출석 word2vec에 없는 단어입니다.
사유서 word2vec에 없는 단어입니다.
평택항 word2vec에 없는 단어입니다.
해항 word2vec에 없는 단어입니다.
묵고 word2vec에 없는 단어입니다.
동아백화점 

In [23]:
# 문장별로 train_test set 분리
from sklearn.model_selection import train_test_split

#num_article = len(train["n_id"].unique())
#trainnp.random.randint(0,num_article,int(0.7*num_article))

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, random_state = 42, test_size = 0.3)
X_title_train, X_title_valid, y_title_train, y_title_valid = train_test_split(title_X, train_y, random_state=42, test_size=0.3)

In [27]:
# 시퀀셜 LSTM 모델
from keras import regularizers
def title_LSTM(vocab_size, title_max_len=1000):
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(vocab_size, 200, weights = [title_embedding_matrix], input_length = title_max_len)) #임베딩 가중치 적용 코드
    model.add(keras.layers.SpatialDropout1D(0.1))
    model.add(keras.layers.LSTM(32, return_sequences=True))
    model.add(keras.layers.LSTM(32))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(16, activation='selu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model

In [28]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("title_hyerim_best_model2.h5",
                                               save_best_only = True)

# 하이퍼파라미터
max_epoch = 5
batch_size = 100

model = title_LSTM(title_vocab_size, title_max_len = title_max_len)
history = model.fit(X_title_train, y_title_train,epochs=max_epoch,
                 batch_size = batch_size, validation_data=(X_title_valid, y_title_valid) , validation_batch_size = batch_size,
                 callbacks = [checkpoint_cb])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 200)           1585200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 20, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 32)            29824     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

-> target변수를 그대로 info로 했을 때, 성능이 좋지 않아 사용하기에 어려움이 있음

# title별로 훈련

In [53]:
group_title = train.groupby(["title"])
title_y = ((group_title["info"].sum())/(group_title["info"].count())).values
title_X=(group_title["info"].count()).index

In [54]:
title_X, title_mecab = text_preprocessing(title_X)
title_X, title_vocab_size, title_vectorizer = text2sequence(title_X, max_len = title_max_len)

vocab_size :  7926


In [56]:
X_title_train, X_title_valid, y_title_train, y_title_valid = train_test_split(title_X, title_y, random_state=42, test_size=0.3)

In [59]:
# 시퀀셜 LSTM 모델
from keras import regularizers
def title_LSTM(vocab_size, title_max_len=1000):
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(vocab_size, 200, weights = [title_embedding_matrix], input_length = title_max_len)) #임베딩 가중치 적용 코드
    model.add(keras.layers.SpatialDropout1D(0.1))
    model.add(keras.layers.LSTM(32, return_sequences=True))
    model.add(keras.layers.LSTM(32))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(16, activation='selu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    return model

In [61]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("title_hyerim_best_model2.h5",
                                               save_best_only = True)

# 하이퍼파라미터
max_epoch = 30
batch_size = 100

model = title_LSTM(title_vocab_size, title_max_len = title_max_len)
history = model.fit(X_title_train, y_title_train,epochs=max_epoch,
                 batch_size = batch_size, validation_data=(X_title_valid, y_title_valid) , validation_batch_size = batch_size,
                 callbacks = [checkpoint_cb])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 200)           1585200   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 20, 200)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 20, 32)            29824     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [63]:
# 성능
model.evaluate(X_title_valid,y_title_valid)

34/34 [==============================] - 0s 7ms/step - loss: 0.0502


0.05022847279906273

-> train, test의 loss격차가 심함